In [3]:
import os
import csv
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor

from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error

#load datasets

airbnb_crime = pd.read_csv(r'C:\Madhuri\projects\ML_project\Test_ML\Nova pasta\cleaned_airnb_crime.csv')


In [4]:
X = airbnb_crime[['latitude', 'longitude', 'minimum_nights', 'number_of_reviews', 'availability_365', 'crime_count',
               'calculated_host_listings_count', 'distance_to_statue_of_liberty', 'distance_to_times_square', 'neighbourhood_group', 'room_type','baths', 'bedrooms', 'rating']]
y = airbnb_crime['price']

In [5]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [6]:
from sklearn.preprocessing import MinMaxScaler

# Select only numeric columns
numeric_features = ['latitude', 'longitude', 'minimum_nights', 
                    'number_of_reviews', 'availability_365', 'crime_count',
                    'calculated_host_listings_count', 
                    'distance_to_statue_of_liberty', 'distance_to_times_square','baths', 'bedrooms', 'rating']

scaler = MinMaxScaler()
scaler.fit(X_train[numeric_features])

X_train_scaled = X_train.copy()
X_test_scaled = X_test.copy()

# Replace numeric columns with scaled versions
X_train_scaled[numeric_features] = scaler.transform(X_train[numeric_features])
X_test_scaled[numeric_features] = scaler.transform(X_test[numeric_features])


In [7]:
# Put transformed data into a DataFrame (only numeric features)
X_train_transformed = pd.DataFrame(scaler.transform(X_train[numeric_features]), columns=numeric_features, index=X_train.index)
X_test_transformed = pd.DataFrame(scaler.transform(X_test[numeric_features]), columns=numeric_features, index=X_test.index)

In [8]:
from sklearn.preprocessing import OneHotEncoder

oh_encoder = OneHotEncoder(drop='first')
oh_encoder.fit(airbnb_crime.loc[X_train.index, ['neighbourhood_group']])

X_train_encoded = pd.DataFrame.sparse.from_spmatrix(
    oh_encoder.transform(airbnb_crime.loc[X_train.index, ['neighbourhood_group']]),
    columns=oh_encoder.get_feature_names_out(),
    index=X_train.index
)
X_test_encoded = pd.DataFrame.sparse.from_spmatrix(
    oh_encoder.transform(airbnb_crime.loc[X_test.index, ['neighbourhood_group']]),
    columns=oh_encoder.get_feature_names_out(),
    index=X_test.index
)

X_train_encoded = pd.concat([X_train_encoded, X_train_transformed], axis=1)
X_test_encoded = pd.concat([X_test_encoded, X_test_transformed], axis=1)

In [10]:
# Define the mapping
room_type_map = {
    'Entire home/apt': 4,
    'Hotel room': 3,
    'Private room': 2,
    'Shared room': 1
}

# Apply the mapping to training and test sets
X_train_label = X_train[['room_type']].replace(room_type_map)
X_test_label = X_test[['room_type']].replace(room_type_map)

# Combine with your other encoded features

X_train_t = pd.concat(
    [X_train_encoded.reset_index(drop=True),
     X_train_label.reset_index(drop=True)],
    axis=1
)

X_test_t = pd.concat(
    [X_test_encoded.reset_index(drop=True),
     X_test_label.reset_index(drop=True)],
    axis=1
)



C:\Users\Naveen\AppData\Local\Temp\ipykernel_11424\3696907422.py:10: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  X_train_label = X_train[['room_type']].replace(room_type_map)
C:\Users\Naveen\AppData\Local\Temp\ipykernel_11424\3696907422.py:11: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  X_test_label = X_test[['room_type']].replace(room_type_map)


In [11]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.neighbors import KNeighborsRegressor


knn = KNeighborsRegressor(n_neighbors=160)
knn.fit(X_train_t, y_train)
y_pred = knn.predict(X_test_t)
knn.score(X_test_t, y_test)

mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)

print("KNN Regression Results")
print("Score: {}".format(knn.score(X_test_t, y_test)))
print("Mean Squared Error: {}".format(mse))
print("Mean Absolute Error: {}".format(mae))
print("Root Mean Absolute Error: {}".format(rmse))
print("R2 score: {}".format(r2))

c:\Users\Naveen\anaconda3\Lib\site-packages\sklearn\utils\validation.py:877: UserWarning: pandas.DataFrame with sparse columns found.It will be converted to a dense numpy array.
  warnings.warn(
c:\Users\Naveen\anaconda3\Lib\site-packages\sklearn\utils\validation.py:877: UserWarning: pandas.DataFrame with sparse columns found.It will be converted to a dense numpy array.
  warnings.warn(


KNN Regression Results
Score: 0.1036533115329511
Mean Squared Error: 79354.85814702035
Mean Absolute Error: 86.62555813953487
Root Mean Absolute Error: 281.69994346293424
R2 score: 0.1036533115329511


c:\Users\Naveen\anaconda3\Lib\site-packages\sklearn\utils\validation.py:877: UserWarning: pandas.DataFrame with sparse columns found.It will be converted to a dense numpy array.
  warnings.warn(
c:\Users\Naveen\anaconda3\Lib\site-packages\sklearn\utils\validation.py:877: UserWarning: pandas.DataFrame with sparse columns found.It will be converted to a dense numpy array.
  warnings.warn(


In [12]:
from sklearn.linear_model import LinearRegression

lr = LinearRegression()
lr.fit(X_train_t, y_train)

y_pred_lr = lr.predict(X_test_t)

# Metrics
mse_lr = mean_squared_error(y_test, y_pred_lr)
mae_lr = mean_absolute_error(y_test, y_pred_lr)
rmse_lr = np.sqrt(mse_lr)
r2_lr = r2_score(y_test, y_pred_lr)   # same as lr.score(X_test_t, y_test)

# Print results
print("Linear Regression Results")
print("Score: {}".format(lr.score(X_test_t, y_test)))
print("Mean Squared Error: {}".format(mse_lr))
print("Mean Absolute Error: {}".format(mae_lr)) 
print("Root Mean Squared Error: {}".format(rmse_lr))
print("R2 score: {}".format(r2_lr))



Linear Regression Results
Score: 0.18829322326466957
Mean Squared Error: 71861.5653447301
Mean Absolute Error: 83.19904074972533
Root Mean Squared Error: 268.0700754368717
R2 score: 0.18829322326466957


c:\Users\Naveen\anaconda3\Lib\site-packages\sklearn\utils\validation.py:877: UserWarning: pandas.DataFrame with sparse columns found.It will be converted to a dense numpy array.
  warnings.warn(
c:\Users\Naveen\anaconda3\Lib\site-packages\sklearn\utils\validation.py:877: UserWarning: pandas.DataFrame with sparse columns found.It will be converted to a dense numpy array.
  warnings.warn(
c:\Users\Naveen\anaconda3\Lib\site-packages\sklearn\utils\validation.py:877: UserWarning: pandas.DataFrame with sparse columns found.It will be converted to a dense numpy array.
  warnings.warn(


In [13]:
from sklearn.tree import DecisionTreeRegressor

dt_reg = DecisionTreeRegressor(
    max_depth=10, # 30, 50, 100 - we can have many more splits when we have continuous variables
    min_samples_split=2,
    max_leaf_nodes=30 # basically equivalent to max_depth but still considered
)


dt_reg.fit(X_train_t, y_train)

y_pred_dt = dt_reg.predict(X_test_t)

r2_dt = dt_reg.score(X_test_t, y_test)

mse_dt = mean_squared_error(y_test, y_pred_dt)
mae_dt = mean_absolute_error(y_test, y_pred_dt)
rmse_dt = np.sqrt(mse_dt)

print("Decision Tree Regression Results")
print("Score: {}".format(dt_reg.score(X_test_t, y_test)))
print("Mean Squared Error: {}".format(mse_dt))
print("Mean Absolute Error: {}".format(mae_dt)) 
print("Root Mean Absolute Error: {}".format(rmse_dt))
print("R2 score: {}".format(r2_dt))

Decision Tree Regression Results
Score: 0.16451471163384657
Mean Squared Error: 73966.71108988627
Mean Absolute Error: 82.37712696968691
Root Mean Absolute Error: 271.96821705832883
R2 score: 0.16451471163384657


c:\Users\Naveen\anaconda3\Lib\site-packages\sklearn\utils\validation.py:877: UserWarning: pandas.DataFrame with sparse columns found.It will be converted to a dense numpy array.
  warnings.warn(
c:\Users\Naveen\anaconda3\Lib\site-packages\sklearn\utils\validation.py:877: UserWarning: pandas.DataFrame with sparse columns found.It will be converted to a dense numpy array.
  warnings.warn(
c:\Users\Naveen\anaconda3\Lib\site-packages\sklearn\utils\validation.py:877: UserWarning: pandas.DataFrame with sparse columns found.It will be converted to a dense numpy array.
  warnings.warn(
c:\Users\Naveen\anaconda3\Lib\site-packages\sklearn\utils\validation.py:877: UserWarning: pandas.DataFrame with sparse columns found.It will be converted to a dense numpy array.
  warnings.warn(


In [14]:
from sklearn.ensemble import RandomForestRegressor

rf_boot = RandomForestRegressor(n_estimators=50, max_depth=10, bootstrap=True, random_state=42)
rf_boot.fit(X_train_t, y_train)

y_pred_rf = rf_boot.predict(X_test_t)

# Metrics
r2_rf = rf_boot.score(X_test_t, y_test)
mse_rf = mean_squared_error(y_test, y_pred_rf)
mae_rf = mean_absolute_error(y_test, y_pred_rf)
rmse_rf = np.sqrt(mse_rf)

# Results
print("Random Forest Regression Results")
print(f"R² Score: {r2_rf:.4f}")
print(f"Mean Squared Error: {mse_rf:.4f}")
print(f"Mean Absolute Error: {mae_rf:.4f}")
print(f"Root Mean Squared Error: {rmse_rf:.4f}")


c:\Users\Naveen\anaconda3\Lib\site-packages\sklearn\utils\validation.py:877: UserWarning: pandas.DataFrame with sparse columns found.It will be converted to a dense numpy array.
  warnings.warn(


Random Forest Regression Results
R² Score: 0.2446
Mean Squared Error: 66877.9548
Mean Absolute Error: 71.3358
Root Mean Squared Error: 258.6077


c:\Users\Naveen\anaconda3\Lib\site-packages\sklearn\utils\validation.py:877: UserWarning: pandas.DataFrame with sparse columns found.It will be converted to a dense numpy array.
  warnings.warn(
c:\Users\Naveen\anaconda3\Lib\site-packages\sklearn\utils\validation.py:877: UserWarning: pandas.DataFrame with sparse columns found.It will be converted to a dense numpy array.
  warnings.warn(


In [16]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
import xgboost as xgb

# Drop columns not useful for prediction (IDs, text)
X = airbnb_crime.drop(columns=['id','price','last_review'])  # keep features
y = airbnb_crime['price']  # target variable

# Handle missing values (replace NaN with median)
X = X.fillna(X.median(numeric_only=True))

# Convert categorical variables to dummies (one-hot encoding)
X = pd.get_dummies(X, drop_first=True)

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# Define model
model = xgb.XGBRegressor(
    n_estimators=500,
    learning_rate=0.05,
    max_depth=6,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42
)

# Train model
model.fit(X_train, y_train)

# Predictions
y_pred = model.predict(X_test)

# Evaluation metrics
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
r2 = r2_score(y_test, y_pred)

print("XGBoost Regression Results")
print(f"RMSE: {rmse:.2f}")
print(f"R² Score: {r2:.3f}")




XGBoost Regression Results
RMSE: 258.32
R² Score: 0.246


In [17]:
# Transform price with log1p (handles zeros safely)
y_log = np.log1p(airbnb_crime['price'])

from sklearn.model_selection import train_test_split

X = airbnb_crime.drop(columns=['id','price','last_review'])
X = X.fillna(X.median(numeric_only=True))
X = pd.get_dummies(X, drop_first=True)

X_train, X_test, y_train, y_test = train_test_split(
    X, y_log, test_size=0.2, random_state=42
)

import xgboost as xgb

model = xgb.XGBRegressor(
    n_estimators=500,
    learning_rate=0.05,
    max_depth=6,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42
)

model.fit(X_train, y_train)

y_pred_log = model.predict(X_test)

# Convert predictions back to original scale
y_pred = np.expm1(y_pred_log)

# If you want to evaluate
from sklearn.metrics import mean_squared_error, r2_score

rmse = np.sqrt(mean_squared_error(np.expm1(y_test), y_pred))
r2 = r2_score(np.expm1(y_test), y_pred)

print(f"RMSE: {rmse:.2f}")
print(f"R² Score: {r2:.3f}")



RMSE: 260.50
R² Score: 0.233
